In [1]:
import os
import tensorflow as tf

import numpy as np
import pandas as pd
from tqdm import tqdm

%load_ext autoreload
%autoreload 2

2024-02-10 20:10:09.571180: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-10 20:10:09.571738: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-10 20:10:09.612450: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
pd.read_csv('/n/holystore01/LABS/iaifi_lab/Lab/quijote_bsq/halos_1234.csv')

,x,y,z,vx,vy,vz,Jx,Jy,Jz,M200c,Rvir
0,523.87921,997.09070,329.35129,-58.62,429.58,121.10,-7.548000e+17,5.696000e+17,-9.084000e+17,4051300000000000,3264.635
1,832.85913,388.96509,288.60141,575.17,1124.71,638.53,-9.456000e+17,-2.231000e+17,4.367000e+17,3363200000000000,3161.956
2,931.41461,481.97626,815.59155,6.48,139.10,20.07,-3.455000e+17,-3.971000e+15,3.417000e+17,3023700000000000,2941.829
3,830.04327,74.93627,536.11200,653.95,41.30,-186.97,-1.212000e+17,1.092000e+18,-8.631000e+16,2881700000000000,2889.544
4,245.69009,977.35895,225.14355,781.71,162.50,181.48,-2.718000e+15,7.374000e+17,6.526000e+17,2784700000000000,2836.105
...,...,...,...,...,...,...,...,...,...,...,...
4995,259.89481,331.97177,943.88849,589.36,-189.89,-11.83,4.858000e+15,2.160000e+16,-1.090000e+15,342250000000000,1462.600
4996,851.34479,954.74152,351.61807,-123.16,-530.66,171.77,-3.083000e+16,1.434000e+16,-7.996000e+15,342250000000000,1421.951
4997,398.50443,974.74878,164.84859,572.37,578.25,611.00,4.723000e+15,-1.924000e+16,3.721000e+16,342250000000000,1458.534
4998,841.52911,938.93311,396.58209,952.90,-270.64,-439.37,8.968000e+15,7.282000e+15,6.092000e+15,342250000000000,1461.585


In [3]:
params = np.loadtxt('/n/holystore01/LABS/iaifi_lab/Lab/quijote_bsq/BSQ_params.txt')
params.shape

params_names = ['Omega_m', 'Omega_b', 'h', 'n_s', 'sigma_8']

In [5]:
def _bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def serialize_example(features, params):
    """
    Creates a tf.train.Example message ready to be written to a file.
    """
    
    feature = {k: _bytes_feature(tf.io.serialize_tensor(tf.cast(v, tf.float32)).numpy()) for k, v in features.items()}
    # feature['params'] = _bytes_feature(tf.io.serialize_tensor(tf.cast(params, tf.float32)).numpy())

    # Params contains 5 values, pick individual values and save using params_names as the key, similar to features above
    for i, param in enumerate(params):
        feature[params_names[i]] = _bytes_feature(tf.io.serialize_tensor(tf.cast(param, tf.float32)).numpy()) 

    example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
    return example_proto.SerializeToString()

params_ary = np.loadtxt('/n/holystore01/LABS/iaifi_lab/Lab/quijote_bsq/BSQ_params.txt')

def process_csv_to_tfrecords(csv_files_path, tfrecords_path, num_tfrecords=10, num_tfrecords_val=1):
    """
    Converts CSV files in a folder to a specified number of TFRecord files.
    """

    # Make tfrecords_path if it doesn't exist
    os.makedirs(tfrecords_path, exist_ok=True)

    csv_files = [os.path.join(csv_files_path, f) for f in os.listdir(csv_files_path) if f.endswith('.csv')]
    num_files = len(csv_files)
    files_per_tfrecord = int(np.ceil(num_files / num_tfrecords))
    
    for i in range(num_tfrecords):
        tfrecord_file = os.path.join(tfrecords_path, f'halos_train_{i + 1}.tfrecord' if i < num_tfrecords - num_tfrecords_val else f'halos_val_{i + 1 - (num_tfrecords - num_tfrecords_val)}.tfrecord')
        with tf.io.TFRecordWriter(tfrecord_file) as writer:
            subset_csv_files = csv_files[i * files_per_tfrecord:(i + 1) * files_per_tfrecord]
            for csv_file in tqdm(subset_csv_files, desc=f'Writing TFRecord {i + 1} / {num_tfrecords}'):
                df = pd.read_csv(csv_file)

                # Get index of csv file, format halos_{index}.csv
                idx = int(csv_file.split('_')[-1].split('.')[0])
                
                features = {col: df[col] for col in df.columns if col != 'id'}
                example = serialize_example(features, params_ary[idx])
                writer.write(example)

csv_files_path = '/n/holystore01/LABS/iaifi_lab/Lab/quijote_bsq'
tfrecords_path = '/n/holystore01/LABS/iaifi_lab/Lab/quijote_bsq_tfrecords'
num_tfrecords = 10  # Specify the number of TFRecord files you want
num_tfrecords_val = 1  # Specify the number of TFRecord files you want for validation

process_csv_to_tfrecords(csv_files_path, tfrecords_path, num_tfrecords, num_tfrecords_val)

Writing TFRecord 10 / 10: 100%|██████████| 1809/1809 [01:46<00:00, 16.97it/s]


In [6]:
def _parse_function(proto, features=['x', 'y', 'z', 'Jx', 'Jy', 'Jz', 'vx', 'vy', 'vz', 'M200c'], 
                    params=['Omega_m', 'Omega_b', 'h', 'n_s', 'sigma_8']):
    
    # Define your tfrecord again. It must match with how you wrote it
    keys_to_features = {k: tf.io.FixedLenFeature([], tf.string) for k in features}
    keys_to_params = {k: tf.io.FixedLenFeature([], tf.string) for k in params}
    
    # Load one example
    parsed_features = tf.io.parse_single_example(proto, keys_to_features)
    parsed_params = tf.io.parse_single_example(proto, keys_to_params)

    # Convert each feature from a serialized string to a tensor and store in a list
    feature_tensors = [tf.io.parse_tensor(parsed_features[k], out_type=tf.float32) for k in features]
    param_tensors = [tf.io.parse_tensor(parsed_params[k], out_type=tf.float32) for k in params]

    # Stack the feature tensors to create a single tensor
    # Each tensor must have the same shape
    stacked_features = tf.stack(feature_tensors, axis=1)  # Creates a [num_points, num_features] tensor
    stacked_params = tf.stack(param_tensors, axis=0)  # Creates a [num_params] tensor

    return stacked_features, stacked_params

In [45]:
from functools import partial

def get_halo_dataset(batch_size=64, features=['x', 'y', 'z', 'Jx', 'Jy', 'Jz', 'vx', 'vy', 'vz', 'M200c', 'Rvir'],
                     params=['Omega_m', 'sigma_8']):

    files = tf.io.gfile.glob(f"/n/holystore01/LABS/iaifi_lab/Lab/quijote_bsq_tfrecords/halos*.tfrecord")
    dataset = tf.data.TFRecordDataset(files)
    num_total = sum(1 for _ in dataset)
    dataset = dataset.map(partial(_parse_function, features=features, params=params))
    dataset = dataset.batch(batch_size)

    return dataset, num_total

In [46]:
features=['x', 'y', 'z', 'Jx', 'Jy', 'Jz', 'vx', 'vy', 'vz', 'M200c', 'Rvir']
params=['Omega_m', 'Omega_b', 'h', 'n_s', 'sigma_8']
batch_size = 64

dataset, num_total = get_halo_dataset(batch_size=64, features=features, params=params)
iterator = iter(dataset)

x, params = next(iterator)
x.shape, params.shape

(TensorShape([64, 5000, 11]), TensorShape([64, 5]))

In [47]:
iterator = iter(dataset)

# First pass: Compute the mean
mean = 0
mean_params = 0
for i in tqdm(range(num_total // batch_size)):
    batch, batch_params = next(iterator)
    batch = tf.cast(batch, tf.float64)
    mean += tf.reduce_mean(batch, axis=(0, 1))
    mean_params += tf.reduce_mean(batch_params, axis=0)
mean /= (num_total // batch_size)
mean_params /= (num_total // batch_size)

# Reset the iterator for the second pass
iterator = iter(dataset)

# Second pass: Compute the variance, then standard deviation
variance = 0
variance_params = 0
for i in tqdm(range(num_total // batch_size)):
    batch, batch_params = next(iterator)
    batch = tf.cast(batch, tf.float64)
    variance += tf.reduce_mean(tf.square(batch - mean), axis=(0, 1))
    variance_params += tf.reduce_mean(tf.square(batch_params - mean_params), axis=0)
variance /= (num_total // batch_size)
variance_params /= (num_total // batch_size)

std = tf.sqrt(variance)
std_params = tf.sqrt(variance_params)

100%|██████████| 283/283 [00:02<00:00, 117.63it/s]


In [50]:
# Make a dict mapping feature names to mean and std
MEAN_HALOS_DICT = {f: m for f, m in zip(features, mean.numpy())}
STD_HALOS_DICT = {f: s for f, s in zip(features, std.numpy())}

MEAN_PARAMS_DICT = {f: m for f, m in zip(params_names, mean_params.numpy())}
STD_PARAMS_DICT = {f: s for f, s in zip(params_names, std_params.numpy())}

# MEAN_HALOS_DICT, STD_HALOS_DICT, MEAN_PARAMS_DICT, STD_PARAMS_DICT

In [10]:
from dataset.dataset import get_halo_dataset

features = ['x', 'y', 'z', 'vx', 'vy', 'vz', 'M200c']
params = ['Omega_m', 'sigma_8']
batch_size = 64

dataset, num_total = get_halo_dataset(batch_size=batch_size, 
                           num_samples=None,  # If not None, will only take a subset of the dataset
                           standardize=True,  # If True, will standardize the features
                           return_mean_std=False,  # If True, will return (dataset, num_total, mean, std, mean_params, std_params), else (dataset, num_total)
                           seed=42,
                           features=features, 
                           params=params)

# Print number of samples
print(f"Number of samples: {num_total}")

Number of samples: 16362


In [13]:
iterator = iter(dataset)

for _ in tqdm(range(num_total // batch_size)):
    x, params = next(iterator)

x.shape, params.shape

  0%|          | 0/255 [00:00<?, ?it/s]

(TensorShape([64, 5000, 7]), TensorShape([64, 2]))